In [2]:
import requests
import json
import time
from verifyers import *
from math import prod
from protocols import *

### prepare data for encryption verification

In [ ]:
public_key = {"y": 0,
            "g": 0,
            "p": 0,
            "q": 0}

choice = {"alphas": [],  
            "betas": []}

commitment = {"A": 0,
                "B": 0}

proof = {"challenge": 0,
            "commitment": commitment,
            "response": 0}

trustees_public_keys = {"ys": [],
                        "g": g,
                        "p": p,
                        "q": q}

answer = {"choice": choice,
        "individual_proofs": (proof,proof) or None,
        "overall_proofs": (proof,proof) or None,
        "trustees_public_keys": trustees_public_keys}

trustee_data = {"public_key":public_key,
                "decryption_proofs": (proof,proof),
                "decryption_factor": 0,
                "alphas": []}

result_data = {"betas": [],
                  "trustees_public_keys": trustees_public_keys,
                  "decryption_factors": [],
                  "result":0}

question = {"answers_data": [answer, answer, answer],
            "trustees_public_keys": trustees_public_keys,
            "trustees_data": trustee_data,
            "result_data": result_data}

election = [question, question, question]

In [ ]:
# sort voters that voted
# get voters uuids

participated_uuids = []
with open(f"data/all_voters_uuids.json") as f:
    json_data = f.read()
voters_uuids = json.loads(json_data)

participated_uuid = 0
for uuid in voters_uuids:
    with open(f"data/ballots/{uuid}.json") as f:
        json_data = f.read()
    ballot = json.loads(json_data)
    vote = ballot['vote']
    if vote is not None:
        participated_uuid += 1
        participated_uuids.append(uuid)
        with open(f'data/participated_ballots/{str(participated_uuid)}.json', 'w') as f:
            # write participated ballot
            json.dump(ballot, f, indent=4)

In [ ]:
# Q1-Q5
all_questions_all_answers = []

with open(f'data/result.json') as f:
    json_data = f.read()
result = json.loads(json_data)

ballot_structure = [6,1,1,1,1,2]
for question_number in range(6):
    number_of_subquestions = ballot_structure[question_number]
    for subquestion_number in range(number_of_subquestions):

        ys = []
        trustees_data = []
        # analyse trustee structure
        with open(f'trustees.json') as f:
            json_data = f.read()

        # Convert the JSON data into a Python object
        trustees = json.loads(json_data)
        l = len(trustees)
        decryption_factors = []
        for i in range(l):
            t = trustees[i]
            public_key = t['public_key']
            p = int(public_key['p'])
            q = int(public_key['q'])
            g = int(public_key['g'])
            y = int(public_key['y'])
            ys.append(y)
            decryption_factor = int(t['decryption_factors'][question_number][subquestion_number])
            decryption_factors.append(decryption_factor)

            trustee_data = {"public_key":public_key,
                                    "decryption_proofs": t['decryption_proofs'][question_number][subquestion_number],
                                    "decryption_factor":int(t['decryption_factors'][question_number][subquestion_number]),
                                    "alphas": []}
            trustees_data.append(trustee_data)

        trustees_public_keys = {"ys": ys,
                                   "g": g,
                                "p": p,
                                "q": q}
        
        result_data = {"betas": [],
                  "trustees_public_keys": trustees_public_keys,
                  "decryption_factors": decryption_factors,
                  "result":int(result[question_number][subquestion_number])}
            
        one_question_all_answers = {"answers_data": [],
                    "trustees_public_keys": trustees_public_keys,
                        "trustees_data":  trustees_data,
                        "result_data": result_data,}
        
        alphas_for_trustee = []
        betas_for_result = []

        for user_id in range(1, 646):
            with open(f'data/participated_ballots/{str(user_id)}.json') as f:
                json_data = f.read()
            ballot = json.loads(json_data)
            vote = ballot['vote']
            answers = vote['answers']
            answer = answers[question_number]
            choices = answer["choices"]
            choice = choices[subquestion_number]
            alpha = int(choice["alpha"])
            beta = int(choice["beta"])
            individual_proofs = answer["individual_proofs"][subquestion_number]
            overall_proofs = answer["overall_proof"]
            alphas_for_trustee.append(alpha)
            betas_for_result.append(beta)

            if question_number == 5:
                # for Q6 overal proof needs both alpha and beta, we will add the later separately
                overall_proofs = None

            alphas = [alpha]
            betas = [beta]
            # answer of one voter to one question
            individual_answer = {"choice": {"alphas": alphas, "betas": betas},
                                "individual_proofs": individual_proofs,
                                "overall_proofs": overall_proofs,
                                "trustees_public_keys": trustees_public_keys}
            # answers of all voters to one question
            one_question_all_answers['answers_data'].append(individual_answer)

        one_question_all_answers['result_data']['betas'] = betas_for_result

        for t in range(3):
            one_question_all_answers['trustees_data'][t]['alphas'] = alphas_for_trustee
        all_questions_all_answers.append(one_question_all_answers)

In [ ]:
# Compose Q6 overal proofs

with open(f'data/result.json') as f:
    json_data = f.read()
result = json.loads(json_data)

ys = []
trustees_data = []
# analyse trustee structure
with open(f'data/trustees.json') as f:
    json_data = f.read()
trustees = json.loads(json_data)
l = len(trustees)

for i in range(l):
    t = trustees[i]
    #print(f'trustee = {t}')
    public_key = t['public_key']
    p = int(public_key['p'])
    q = int(public_key['q'])
    g = int(public_key['g'])
    y = int(public_key['y'])
    ys.append(y)

trustees_public_keys = {"ys": ys,
                            "g": g,
                        "p": p,
                        "q": q}

all_questions_all_answers_6 = []

question_number = 5
individual_answers_q6 = []

for user_id in range(1, 646):

    with open(f'data/participated_ballots/{str(user_id)}.json') as f:
        json_data = f.read()

    ballot = json.loads(json_data)
    vote = ballot['vote']
    answers = vote['answers']
    answer = answers[question_number]
    choices = answer["choices"]
    alpha1 = int(choices[0]["alpha"])
    alpha2 = int(choices[1]["alpha"])
    beta1 = int(choices[0]["beta"])
    beta2 = int(choices[1]["beta"])
    overall_proofs = answer["overall_proof"]

    alphas = [alpha1, alpha2]
    betas = [beta1, beta2]

    individual_answer = {"choice": {"alphas": alphas, "betas": betas},
                        "individual_proofs": None,
                        "overall_proofs": overall_proofs,
                        "trustees_public_keys": trustees_public_keys}
    
        # answers of all voters to one question

    individual_answers_q6.append(individual_answer)


In [ ]:
# Add Q6 overal proofs to the rest of the questions
all_questions_all_answers[11]['answers_data'] += individual_answers_q6

In [3]:
# convert all strings to integers
def conversion(d):
    if isinstance(d, list):
        return [conversion(x) for x in d]
    elif isinstance(d, str):
        return int(d)
    elif isinstance(d, dict):
        return {k: conversion(v) for k, v in d.items()}
    elif isinstance(d, int):
        return d
    elif d is None:
        return d
    else:
        raise TypeError(f'Unknown type : {type(d)} d = {d}')


with open(f'data/election_data_restructured.json') as f:
    json_data = f.read()
all_questions_all_answers = json.loads(json_data)

all_questions_all_answers_int = conversion(all_questions_all_answers)

In [4]:
# save the data in new structure
with open('data/election_data_restructured_int.json', 'w') as f:
    json.dump(all_questions_all_answers_int, f,indent=4)